In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-03-15"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
16306,2024-03-15,Turquia Tbl,14:00,Kocaeli BŞB Kağıtspor,Final Spor,1.73,2.01,159.5,1.87,1.79,-2.5,2.01,2.39,Q3TGkPdN,0.578035,0.497512,0.534759,0.558659,0.075547,225.614,192.495763,0.853208,1.2,1.643168,1.369306,126.00,3.138,2.458733,0.783535,-3.0,163.506,32.452523,0.198479,2.4,1.341641,0.559017,137.28,1.974,0.531818,0.269411,19.0,72,88,1.75,1.56,195.822,194.398,0.105877,0.030912,0.122137,-1.64,-0.328,-2.225610,0.000000,0.0,0.000000,-2.95,-0.590,-1.711864,0.000000,0.0,0.000000
16307,2024-03-15,Uruguai Liga Uruguaia,21:00,Cordon,Nacional,2.88,1.38,150.5,1.85,1.85,0.0,0.00,0.00,zqSEDls7,0.347222,0.724638,0.540541,0.540541,0.071860,223.746,159.398546,0.712408,2.4,1.341641,0.559017,129.75,2.992,2.117704,0.707789,34.0,107.906,30.770214,0.285158,1.2,1.643168,1.369306,96.94,1.504,0.503269,0.334621,-4.0,75,74,1.73,1.31,158.898,155.532,0.497963,0.000000,NaN,0.62,0.124,15.161290,0.000000,0.0,0.000000,0.02,0.004,95.000000,0.599288,0.7,0.100712
16308,2024-03-15,Uruguai Liga Uruguaia,22:00,Aguada,Hebraica y Macabi,1.47,2.56,174.5,1.87,1.79,0.0,0.00,0.00,nJVIC8dD,0.680272,0.390625,0.534759,0.558659,0.070897,136.864,24.167157,0.176578,3.0,0.000000,0.000000,103.32,1.432,0.307766,0.214920,84.0,141.912,57.950720,0.408357,2.4,1.341641,0.559017,103.23,1.596,0.689043,0.431731,31.0,82,93,1.26,1.11,129.442,167.766,0.382504,0.030912,NaN,0.25,0.050,9.400000,0.746547,0.9,0.153453,0.48,0.096,16.250000,0.543338,0.5,-0.043338
16309,2024-03-15,Uruguai Liga Uruguaia,19:00,Goes,Bigua,2.10,1.68,168.5,1.87,1.79,1.5,1.96,1.53,8IV9xq3l,0.476190,0.595238,0.534759,0.558659,0.071429,243.434,127.793204,0.524960,1.8,1.643168,0.912871,176.64,2.854,1.642842,0.575628,41.0,158.758,66.113782,0.416444,1.2,1.643168,1.369306,252.45,1.996,0.946113,0.474005,5.0,92,85,1.92,2.97,197.746,179.496,0.157135,0.030912,0.174244,1.42,0.284,3.873239,0.000000,0.0,0.000000,-3.51,-0.702,-0.968661,0.000000,0.0,0.000000
16310,2024-03-15,Uruguai Liga Uruguaia,21:00,Malvin,Trouville,1.30,3.34,165.5,1.81,1.86,0.0,0.00,0.00,OlOKLBZg,0.769231,0.299401,0.552486,0.537634,0.068632,134.342,9.262123,0.068944,1.2,1.643168,1.369306,118.68,1.684,0.261209,0.155112,-7.0,176.936,90.561121,0.511830,1.2,1.643168,1.369306,123.28,2.228,1.027385,0.461124,7.0,92,92,1.29,1.34,135.604,267.686,0.621766,0.019267,NaN,-2.26,-0.452,-0.663717,0.701863,0.6,-0.101863,-1.01,-0.202,-11.584158,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16443,2024-03-15,França Pro B,16:00,La Rochelle,Chalons-Reims,1.30,3.31,152.5,1.85,1.85,0.0,0.00,0.00,p8nFf9B1,0.769231,0.302115,0.540541,0.540541,0.071346,114.666,23.400830,0.204078,2.4,1.341641,0.559017,105.07,1.550,0.400687,0.258508,53.0,130.736,49.799148,0.380914,1.8,1.643168,0.912871,108.75,1.626,0.367872,0.226244,35.0,79,75,1.33,1.45,98.842,163.342,0.616609,0.000000,NaN,1.14,0.228,1.315789,0.000000,0.0,0.000000,1.19,0.238,9.705882,0.000000,0.0,0.000000
16444,2024-03-15,França Pro B,16:00,Lille,Antibes,1.56,2.32,154.5,1.85,1.85,0.0,0.00,0.00,2qcAekde,0.641026,0.431034,0.540541,0.540541,0.072060,140.322,47.868423,0.341133,1.2,1.643168,1.369306,184.03,1.964,

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
16327,12:30,Romênia Divisão A,Dinamo Bucharest,CSM Constanta,2.03,Back Home
16341,18:00,Eua Ncaa,Bowling Green,Kent State,2.10,Back Home
16343,22:30,Eua Ncaa,Virginia,N.C. State,1.67,Back Home
16361,19:30,Eua Ncaa,Illinois,Ohio State,1.55,Back Home
16405,16:30,Europa Euroliga,Bayern,Zalgiris Kaunas,1.61,Back Home
16419,16:30,Itália Série A2,Cividale,Basket Torino,1.66,Back Home
16425,23:15,México Cibacopa,Venados de Mazatlan,Zonkeys de Tijuana,1.78,Back Home
16433,16:00,Europa Liga Bnxt,Liege,Zwolle,1.56,Back Home
16435,16:30,Europa Liga Bnxt,Spirou Charleroi,Den Bosch,2.34,Back Home
16440,16:00,França Lnb,Roanne,Gravelines-Dunkerque,1.65,Back Home
